# Практическое задание 6: Градиентный бустинг

## Подготовка рабочей среды
Сначала установим нужные нам версии библиотек.

После установки нужных версий, **возможно,** нужно перезагрузить среду (runtime), но скорее всего вам это не понадобится

In [1]:
!pip install gdown numpy pandas scikit-learn xgboost catboost lightgbm hyperopt matplotlib seaborn

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\jingni\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Используемые библиотеки

В этом задании могут понадобится четыре библиотеки (помимо sklearn), а именно:

**XGBoost**: Документация [здесь](https://xgboost.readthedocs.io/en/stable/).<br />
**LightGBM**: Документация [здесь](https://lightgbm.readthedocs.io/en/latest/index.html). Также дополнительно про установку [тут](https://pypi.org/project/lightgbm/).<br />
**Catboost**: Документация [здесь](https://catboost.ai/en/docs/). Можно найти также некоторую информацию на русском [тут](https://habr.com/ru/company/otus/blog/527554/).<br />
**HyperOpt**: Документация [здесь](http://hyperopt.github.io/hyperopt/). <br />

<font color='red'>**Внимание!**</font> Вникать и подробно читать документацию к каждой библиотеке нет необходимости! Достаточно обращаться туда для нахождения примеров.


## Напоминание, как правильно перебирать параметры


1. <font color='green'>**learning_rate**</font> $-$ **темп обучения** нашего метода. Для этого метода сетка перебора должна быть логарифмической, т.е. перебирать порядковые значения (к примеру, `[1e-3, 1e-2, 1e-1, 1]`). В большинстве случаев достаточно перебрать значения от `1e-5 до 1`.<br />

2. <font color='green'>**max_depth**</font> $-$ **максимальная глубина деревьев** в ансамбле. Вообще говоря, эта величина зависит от числа признаков, но обычно лучше растить небольшие деревья. К примеру, библиотека `CatBoost`, которую мы будем исследовать сегодня, рекомендует перебирать значения до `10` (и уточняется, что обычно оптимальная глубина лежит `от 6 до 10`).<br />

3. <font color='green'>**subsample**</font> $-$ **объем выборки**, использующийся для обучения отдельного дерева, лежит в интервале `(0, 1]`. Перебирать стоит хотя бы с шагом `0.25` <br />

4. <font color='green'>**n_estimators**</font> $-$ **количество деревьев** в ансамбле. Обычно стоит перебирать с каким-то `крупным шагом` (можно по логарифмической сетке). Здесь важно найти баланс между производительностью, временем обучения и качеством. Обычно `нескольких тысяч` деревьев бывает достаточно.<br />

<font color='red'>**NB!**</font> Учтите, что в реальных задачах необходимо следить за тем, что оптимальные значения параметров не попадают на границы интервалов, т.е. что вы нашли хотя бы локальный минимум. Если вы перебрали значения параметра от 1 до 10 и оказалось, что 10 $-$ оптимальное значение, значит следует перебрать и бОльшие числа, чтобы убедиться, что качество не улучшается дальше (или по крайней мере убедиться, что рост качества сильно замедляется и на сильное улучшение рассчитывать не стоит.


### HyperOpt

Обычнно перебор параметров и поиск по сетке это самая скучная часть работы, поскольку занимает много времени, но не гарантирует воспроизведение результата при небольшом изменении датасета, да и сетку надо переосмысливать при каждом обновлении.

Но этого можно избежать, поскольку есть библиотека, которая всё сделает за нас! Нашего спасителя зовут `HyperOpt`. На первый взгляд hyperopt делает всё то же самое, что и grid search, а именно перебирает параметры. По факту же hyperopt превращает это в задачу оптимизации, используя некоторые эвристики для ускорения сходимости процесса. К тому же, он требует лишь информацию о границе интервалов, а не сами сетки. В теории это должно помочь нам добиться лучших результатов за более короткое время. 

Примерный порядок действий:
1. Взять `любую библиотеку градиентного бустинга`.
2. Составить `сетку перебора в hyperopt`, включающую параметры n_estimators, max_depth, subsample и learning_rate в hyperopt. Вам могут понадобиться такие типы данных, как `hp.choice, hp.qloguniform, hp.uniform и hp.quniform`(можно также пользоваться np.arange). Также для округления значения типа float до целых чисел (4.0 $\to$ 4) можно использовать `scope.int`.
3. `Реализовать функцию`, которая принимает на вход словарь параметров для регрессора, и при помощи CV оценивает его качество на датасете (можно воспользоваться cross_val_score, а для ускорения поставить cv=3). 
4. Создать объект `trials=Trials()`, который будет хранить информацию о процессе оптимизации. Используя функцию fmin, можно `оптимизировать функцию`. Можно установить algo=tpe.suggest, trials=trials и max_evals, по крайней мере, 50. verbose=1 позволит видеть прогресс-бар по типу tqdm.

# Предсказание зрительских симпатий

----------------------------------------------
<font color="white" style="opacity:0.2023"></font>

В некотором царстве, некотором государстве была развита кинопромышленность. Новые фильмы в этом государстве показывают по интернету, а пользователи после просмотра могут дать фильму некоторую "награду". Наша цель $-$ предсказать число наград для фильма.

### Описание данных

В нашем распоряжении имеются следующие данные:

- **awards** $-$ количество наград, полученных фильмом от пользователей (целевое значение)  
- **potions** $-$ количество магических зелий, потраченных на создание спец-эффектов  
- **genres** $-$ жанры созданного фильма  
- **questions** $-$ количество вопросов, заданных пользователями на соответствующих форумах об этом фильме до премьеры  
- **directors** $-$ режиссеры фильма (если неизвестны, то `unknown`)  
- **filming_locations** $-$ области, в которых снимался фильм  
- **runtime** $-$ продолжительность фильма в некоторых единицах, принятых в этом государстве  
- **critics_liked** $-$ количество критиков из 100, присудивших награды фильму на предварительных закрытых показах  
- **pre-orders** $-$ количество зрителей, заранее купивших билеты на первый показ  
- **keywords** $-$ ключевые слова, описывающие содержание фильма
- **release_year** $-$ год, во котором фильм был показан (конечно, в летоисчислении этого государства)

Следующие поля появляются несколько раз с разными значениями `i`:

- **actor_i_known_movies** $-$ количество известных фильмов актера `i` (i от 0 до 2)
- **actor_i_postogramm** $-$ количество подписчиков в социальной сети "по сто грамм" актера `i` (i от 0 до 2)
- **actor_i_gender** $-$ пол актера `i` (i от 0 до 2)
- **actor_i_age** $-$ возраст актера `i` (i от 0 до 2)

### Формат данных

Данные разбиты на три части: тренировочная часть, публичные тестовые команды (которые вы можете скачать) и приватные тестовые файлы. 

Данные хранятся в виде JSONL-файлов, то есть файлов, в которых каждая строка $-$ это JSON. Каждая строка соответствует одному объекту в датасете. Ключи этого JSON соответствуют названиям переменных. В примере решения Вы можете увидеть как читать такие файлы без проблем.

**Важно!** В тестовом файле сохраняйте даннные в том же порядке, в котором они записаны в файл!

---
Вот по [этой ссылке](https://drive.google.com/file/d/1wCOyYTt-n1UU1Snf5O4ExVUuvkhkI3y_/view?usp=sharing) хранится ZIP-архив с публичными данными и шаблоном решения.

In [1]:
# при работе в Colab'е можно использовать эту ячейку для скачивания
!gdown 1wCOyYTt-n1UU1Snf5O4ExVUuvkhkI3y_

Downloading...
From: https://drive.google.com/uc?id=1wCOyYTt-n1UU1Snf5O4ExVUuvkhkI3y_
To: c:\Users\jingni\Documents\Codes\sch57\i-love-crispy-laptops\ML\ML6_Boosting2\solution_template.zip

  0%|          | 0.00/467k [00:00<?, ?B/s]
100%|██████████| 467k/467k [00:01<00:00, 342kB/s]
100%|██████████| 467k/467k [00:01<00:00, 341kB/s]


#### Устройство архива

```
solution_template/
│
├── run.py
│   → Основной скрипт запуска решения. 
│
├── awards_prediction.py
│   → Модуль с логикой предсказаний (обучение модели, обработка данных,
│     применение ML-алгоритма и т.п.). Может содержать класс или функции для
│     подготовки данных и построения прогноза.
│
└── public_tests/
    ├── 01_boosting_movies_gt/
    │   └── target.json
    │       → Файл с эталонными значениями для публичного теста.
    │
    └── 01_boosting_movies_input/
        ├── test/
        │   └── test.jsonl
        │       → Публичный тестовый набор данных в формате JSON Lines.
        │
        └── train/
            └── train.jsonl
                → Публичный обучающий набор данных в формате JSON Lines.
```


### Решение
**<font color='red'>Внимание!</font>** Подбирать оптимальные параметры стоит **только** на локальном компьютере / в Google Colab!
В решении Вы должны использовать регрессор с оптимальными параметрами, которые вы нашли путём перебора по сетке.

В шаблонном файле `awards_prediction.py` Вы должны реализовать функцию `train_model_and_predict`, которая получает на вход папку для обучения и теста. На обучении вы обучаете ваш алгоритм, а затем возвращаете предсказания значений `awards` для всех фильмов из теста. Предсказания должны быть расположены в том же порядке, в котором они находятся в тесте (то есть, не примените где-то случайно `shuffle`).

В этом файле вы можете создавать любые дополнительные функции и методы, которые нужны вам для решения. Главное – сохранить интерфейс функции `train_model_and_predict`.

### Советы по решению
В этом задании Вы можете добиться лучшего качества при помощи:
- Предобработки датасета, выбора категориальных переменных и дополнительной фильтрации.
- Выбора лучшего метода обучения и подбора оптимальных параметров с использованием кросс-валидации.

**<font color='red'>Внимание!</font>** Учтите, что при OHE кодировании признаки на обучении и тестировании должны совпадать! Если вы примените простое `.get_dummies()` или что-то подобное, то признаки на трейне и тесте получатся разные! Так что вам, вероятно, придётся придумать способ для того, чтобы сохранить их :)  

**<font color='red'>Внимание!</font>** Нельзя исключать вероятность того, что злые силы добавили в наш датасет пропуски, поэтому лучше лишний раз как-нибудь заполнить пропущенные значения.

**Подсказка**: для работы с текстом можно воспользоваться методом TF-IDF (ключевые слова: TfIdfTransformer). Также может быть полезен CountVectorizer. Только учтите, что никто не гарантирует улучшение результата с использованием данных методов  ;)

### Разрешенные методы и библиотеки
В качестве метода обучения предлагается использовать **любой регрессор**, основанный на градиентном бустинге деревьев. Разрешается пользоваться библиотеками sklearn, xgboost, lightgbm, catboost.

**Жесткого требования использовать градиентный бустинг нет!** Градиентный бустинг является одним из лучших методов обучения на сегодняшний день, поэтому будет даже интересно, удастся ли кому-то получить макс. балл альтернативными методами.

Также разрешается пользоваться библиотекой hyperopt для подбора параметров модели (параметры стоит подбирать локально, а в систему загружать решение с подобранными параметрами).

### Оценивание

В качестве метрики качества используется значение MAE.

Баллы выставляются по следующим правилам:
- 9 баллов: $\text{MAE} ∈ [0, 2050]$,
- 7 баллов: $\text{MAE} ∈ (2050, 2100]$,
- 5 баллов: $\text{MAE} ∈ (2100, 2150]$,
- 3 баллов: $\text{MAE} ∈ (2150, 2200]$,
- 1 балла: $\text{MAE} ∈ (2200, 2300]$,
- 0 баллов: $\text{MAE} ∈ (2300, +∞]$

Значение MAE будет посчитано раздельно на публичной и приватной выборках. Количество полученных баллов на приватной выборке будет дополнительно умножено на 2. Таким образом за это задание Вы можете получить до 27 баллов (9 на публичной и 18 на приватной выборках).

### Тестирование
Скачайте ZIP-архив c шаблоном решения и разархивируйте его. Далее следуйте инструкциям по запуску тестирования.

Тесты запускаются внутри папки с шаблоном через терминал с помощью команды:
```bash
python run.py
```

Учтите, что после запуска скрипта будет создано несколько дополнительных файлов и директорий (это
связано с работой тестирующей системы).

Если всё верно, то Вы увидите что-то вроде `Mark: 1 OK, mae = [2287.341688095019]`, т.е. Вашу оценку и значение MAE на публичном датасете.

## **<font color='orange'>Задание (27 баллов)</font>**
**Данные**: датасет с наградами за фильмы

**Метрика**: MAE  

**Цели**: В данном задании следует выполнить следующие пункты:  
1. Взять `любую библиотеку`.
2. Используя предложенный датасет, `обучить регрессор` для предсказания awards (предоставляем полную свободу в настройках и выборе методов).
3. Получить качество больше порогового значения.

In [ ]:
## your efficient code here

In [4]:
results = {
  "GBR": [
    7274.681968763821,
    10492.398381395566,
    4146.36285377815,
    16866.744290687093,
    1546.1141196618487,
    10657.865737078622,
    2959.429418386007,
    5662.6569976473775,
    4853.572303053435,
    5925.225680740956,
    6957.795038542266,
    22219.87421694568,
    13369.261660268687,
    5388.065786683827,
    10883.998261020144,
    7404.97114470035,
    7475.957729050238,
    1920.4048075973008,
    1690.7849083481317,
    2849.1916983691426,
    11862.083260740921,
    11623.754192461132,
    26792.369812833775,
    9373.586008543274,
    8228.878969384894,
    12556.726626106536,
    7450.833417820253,
    7472.911758208412,
    4189.742517838717,
    10768.0216928213,
    7176.563691670831,
    14017.614758458893,
    6265.2317199350955,
    14081.691626531197,
    13790.140628698568,
    6532.164369175753,
    3239.297692065167,
    23633.75065511416,
    5274.020020436372,
    9844.929104801886,
    8812.24377961573,
    17638.987082018466,
    11139.392555926739,
    9582.699104307701,
    5950.278460873276,
    19849.379561360307,
    4763.102439509094,
    5009.35178172965,
    9076.840239662579,
    13842.802440336944,
    4362.001138006487,
    9679.154750568669,
    6814.826161920807,
    6215.1805258647455,
    28022.62945096844,
    11156.456874347763,
    5278.4520766889655,
    9772.609477098527,
    2149.043830942788,
    2241.043280052813,
    2563.6975114034003,
    2881.699515483661,
    5883.242275100245,
    16119.498526936199,
    1984.2218118194426,
    9457.736426991296,
    6345.086795090896,
    1826.4719794546993,
    2716.153394622582,
    5629.570788135439,
    8686.160080977885,
    6066.152246232048,
    11725.947636006249,
    6989.060427688117,
    2428.3943598299147,
    11615.560614666521,
    3987.9499299277813,
    2515.908003960591,
    10723.021356571631,
    2376.367878596525,
    18866.673888067897,
    4892.604196420594,
    7075.679944206999,
    6738.278872148133,
    9716.61857043018,
    15203.931637342295,
    7638.3317144530365,
    14551.484304890238,
    7377.568934093496,
    9648.490067144012,
    5590.14347903172,
    4037.112559025779,
    12622.427223751343,
    5561.932695264603,
    12475.087654584644,
    5570.707643642187,
    6578.520961288521,
    4575.777982744548,
    16808.978438417704,
    6382.765825866404,
    2774.566025934412,
    7367.2662657644505,
    7460.7160721449145,
    10624.430127895213,
    10483.703366031057,
    8321.369239974034,
    5408.369512556401,
    6924.627707596071,
    6415.173842442332,
    5581.871277121236,
    1452.7770363503132,
    8575.096304940753,
    16420.6217087341,
    7658.466859390427,
    2255.8016174915015,
    6041.12497627543,
    5780.38513886052,
    8131.818160563114,
    11399.517430510434,
    13671.746265950851,
    6532.352697548252
  ],
  "XGB": [
    7172.54052734375,
    9463.44140625,
    4425.02783203125,
    15965.478515625,
    1089.46533203125,
    11303.1484375,
    2866.052734375,
    5506.38818359375,
    5080.693359375,
    5415.24365234375,
    5538.2900390625,
    24251.361328125,
    13915.1220703125,
    5451.45166015625,
    11036.775390625,
    7813.732421875,
    7659.5,
    1707.453125,
    1604.1600341796875,
    1583.437255859375,
    11931.232421875,
    12829.033203125,
    29598.171875,
    9189.11328125,
    9520.84765625,
    13873.70703125,
    7731.59716796875,
    6801.4853515625,
    3556.421630859375,
    12507.0576171875,
    7320.6162109375,
    14725.7880859375,
    5899.88330078125,
    12966.9326171875,
    12963.08984375,
    6939.85595703125,
    2625.920654296875,
    28180.419921875,
    4983.46630859375,
    11578.3232421875,
    9612.1875,
    20216.552734375,
    13314.685546875,
    9887.740234375,
    6470.76953125,
    20003.115234375,
    3613.283447265625,
    5480.916015625,
    10071.1494140625,
    14957.083984375,
    3971.0810546875,
    10275.9619140625,
    8709.4521484375,
    6103.86767578125,
    28244.1640625,
    11053.6005859375,
    5661.1806640625,
    10917.4267578125,
    1510.1258544921875,
    1766.3782958984375,
    2351.260498046875,
    2542.53076171875,
    6526.77978515625,
    16958.494140625,
    1460.2254638671875,
    9453.54296875,
    5311.38720703125,
    1163.393310546875,
    2191.45947265625,
    5232.27734375,
    8338.5390625,
    6206.67138671875,
    10562.58984375,
    7115.1611328125,
    998.85546875,
    11618.6904296875,
    3076.660400390625,
    1914.2025146484375,
    10951.25390625,
    2026.2489013671875,
    20114.51953125,
    5629.59765625,
    6826.31640625,
    5191.36572265625,
    10388.130859375,
    14674.455078125,
    8423.9921875,
    14924.607421875,
    7532.69140625,
    10636.2705078125,
    5320.2607421875,
    3789.878173828125,
    12218.0849609375,
    4443.462890625,
    13338.6494140625,
    5624.02490234375,
    7163.79833984375,
    4109.93359375,
    19112.90234375,
    7465.0732421875,
    2230.02734375,
    6250.90576171875,
    8060.5341796875,
    10416.955078125,
    10819.662109375,
    7735.59423828125,
    5633.3662109375,
    5222.71875,
    6928.72900390625,
    5600.17529296875,
    649.0216064453125,
    9758.2890625,
    19006.42578125,
    7003.65771484375,
    1711.4139404296875,
    6210.802734375,
    6361.5791015625,
    7339.6845703125,
    11371.763671875,
    13779.7197265625,
    6330.30224609375
  ],
  "KNN": [
    4180.37778211199,
    12343.244342209553,
    3363.202156594067,
    14964.031044206426,
    1820.7070090175887,
    11010.121634913445,
    2354.7674195140976,
    4518.387587379705,
    9103.365358389505,
    5577.686867043419,
    7700.737367171875,
    16320.382835046204,
    14010.167532001245,
    8189.22622463293,
    10933.077625092817,
    8490.989841995894,
    5199.641610842826,
    2304.7847895330183,
    1185.7648798162854,
    3063.0534857362513,
    11419.536445083888,
    11351.744869226843,
    18251.83839775981,
    9947.442691145325,
    10841.811675985453,
    18479.156785008003,
    5534.810116533579,
    8647.372386129944,
    4578.243534506157,
    12797.979032838764,
    7417.192497193039,
    10865.020058083062,
    5831.206494599743,
    9179.674368053436,
    11448.605510371508,
    5277.53076955543,
    1120.2212444044662,
    17824.288407841195,
    7978.51444895001,
    8382.747561790684,
    8436.436892688314,
    15544.301036902967,
    9952.431590489898,
    13287.962732732902,
    7537.720737173326,
    14648.798985008034,
    3519.8083546416287,
    1813.2384093564506,
    14893.275138641651,
    7900.550504667983,
    6064.373535862362,
    10283.139519377255,
    5352.707534346309,
    8238.667190382494,
    27785.317556398302,
    12156.328236537907,
    4756.19929358714,
    8629.334054105091,
    595.3325076305666,
    1175.841753839623,
    3944.570407985053,
    4746.778918784557,
    5540.256496128315,
    15044.552798371054,
    1274.6179969042616,
    8347.56981403011,
    5020.803022755715,
    1696.0170352376617,
    2974.0896280371567,
    5366.879773584405,
    6931.271238722413,
    4757.206099080044,
    7607.979709821289,
    7664.776637418512,
    2481.342501119673,
    8806.2041348199,
    3684.9303616299017,
    3454.7398752671966,
    6442.165206328836,
    2046.9475020050193,
    18925.643619751267,
    7990.268662479022,
    7830.755004096235,
    7671.4721928535655,
    19349.929546066527,
    18122.46606920864,
    5907.978357213131,
    12547.265200505783,
    7587.763258679248,
    14873.60653578629,
    5328.11174894347,
    8013.877291656129,
    16535.976742481093,
    10427.800361905232,
    9221.122171608422,
    4530.55749754175,
    7914.960102127214,
    7597.905143091331,
    15394.52930619846,
    7259.213628206428,
    3633.0172202484678,
    6332.167927232801,
    5423.69416130858,
    9185.028813351182,
    11652.256369623574,
    5485.413095565839,
    4179.403351353282,
    5655.469396412043,
    6834.7755642826105,
    7184.962879027054,
    1425.8302498812002,
    8096.785839744608,
    15284.646129592415,
    6624.881936057358,
    1765.872096637738,
    6559.3886611448515,
    7216.37926980447,
    9096.659931874356,
    10050.727501765608,
    12214.28769348884,
    5893.8322168712475
  ],
  "LGBM": [
    8081.3330621231435,
    9598.124873466111,
    4687.122091038923,
    15308.274728899687,
    802.6330511153068,
    10994.11269476018,
    3411.5843829126975,
    5897.595200276146,
    5373.831798148832,
    5257.779075507736,
    6351.463688807472,
    23396.135790277556,
    14060.62070372157,
    4577.435631805587,
    10852.060469239928,
    8247.817566925807,
    7740.893446221754,
    1907.3985549785004,
    2205.649528241288,
    1783.9333149524698,
    10714.634433150559,
    12656.546621915233,
    29284.341784266428,
    9859.628265750793,
    9086.307715255947,
    12101.269348772792,
    7935.477881682398,
    7162.605050144975,
    3303.990246680364,
    10252.740402246209,
    7912.249645751103,
    13835.089779050735,
    6423.858062857057,
    10189.48715107021,
    12769.829765695004,
    6333.35448171391,
    1938.2987444293651,
    26585.360944521617,
    5134.648340279401,
    11348.27870064009,
    10010.30208170298,
    19340.31740255465,
    12586.153089113535,
    9801.039400838321,
    6447.069381460635,
    18200.330763280308,
    2826.518592606001,
    5477.693034956925,
    10144.60938236367,
    14307.90888951061,
    3818.1497162261608,
    9973.959492965916,
    8596.944030512715,
    6864.750414393617,
    28414.14845293821,
    11168.57247372658,
    6316.544210660329,
    11175.043793458572,
    986.7572482891916,
    2428.173934400059,
    2516.489239711174,
    2300.7448484767197,
    5822.098720787855,
    16656.97051171071,
    1602.5898004857759,
    8626.059108755373,
    5725.427769385404,
    658.9813078402473,
    2097.612567301293,
    6148.468282597257,
    8435.670460181002,
    6588.969729366067,
    11238.946864864149,
    7041.86584751333,
    509.47317665343616,
    11925.573448742829,
    2603.6754001822555,
    2183.212465294732,
    10481.2204475575,
    2212.7724752296194,
    21376.83924847465,
    6586.787578356792,
    6436.090651781516,
    4786.760144525993,
    11161.230123755655,
    14671.537543575743,
    8393.095020980089,
    14363.120498902856,
    6975.202994900911,
    10907.799661173494,
    5068.341057409971,
    4083.347206103728,
    11863.080510111831,
    6337.00171956873,
    13052.15393570713,
    6379.438774418571,
    6317.195874453071,
    5288.697179373646,
    17576.221210781412,
    6015.794081468798,
    1706.7923185833608,
    6896.7110317206425,
    7791.433530265541,
    10419.25840214217,
    11472.818823014659,
    7584.367765562424,
    4980.386303446796,
    6499.295504795846,
    6501.795628697577,
    5276.965374206939,
    846.9927433858409,
    8252.990776536662,
    19384.835079990396,
    6799.201919180881,
    1153.3585600752874,
    6775.109103798468,
    6910.871650197635,
    8737.420716681429,
    11159.382640621601,
    13194.840635141125,
    6789.08386942299
  ],
  "Ridge": [
    10138.028004991611,
    9095.348508198498,
    4493.4965624870165,
    14558.54559007809,
    2850.4900775125498,
    7919.964203779746,
    3963.341211562125,
    6726.004517286638,
    5870.5709031932565,
    6139.707017691355,
    7036.011572745105,
    21957.39942729309,
    12714.25089746117,
    5484.84616551159,
    8815.195117926472,
    5989.539575708128,
    8227.799178924863,
    2429.282402789504,
    3282.0059009263277,
    4012.6813371122407,
    9489.899441628731,
    9456.087867547503,
    25664.007083352095,
    7539.060949210463,
    6497.949504293167,
    12530.489342665926,
    8343.968874013015,
    5931.609384844756,
    5905.5769490565945,
    7544.7342169610565,
    6300.565488462409,
    13590.856905371118,
    5715.7310115587825,
    13554.896613191337,
    13360.698232418425,
    6381.611635920337,
    4424.072781283045,
    21367.191485170497,
    4840.246020788742,
    10542.029600900278,
    7627.922495034531,
    17340.26453803599,
    9023.8855493503,
    8369.821903840872,
    5015.791078183785,
    19309.930206728615,
    6584.617787933705,
    5836.996402451892,
    8325.849300991766,
    12426.595231947314,
    5289.8841156028575,
    7087.6085422009855,
    5768.980848472354,
    5699.245256998365,
    32137.823400808884,
    10275.671673475576,
    5258.826565201867,
    8977.077222867978,
    3272.7132373559725,
    2066.856451721673,
    3621.819014946188,
    3533.7070136399198,
    4957.04188448684,
    15885.245368903488,
    3227.9219250033057,
    7339.5074040466425,
    6078.004219531344,
    3370.360928531023,
    3744.3218162784906,
    5244.204035315757,
    7214.0753098647765,
    7346.618272482527,
    11825.247937340224,
    5886.582898913803,
    3330.155828320654,
    10822.974857913781,
    5055.875695825591,
    3919.821718329782,
    9843.027280295435,
    3640.4117644348225,
    14471.587040359394,
    4292.137715608595,
    6922.549010455361,
    6271.299553299439,
    11652.638479650668,
    14272.04939701641,
    7205.508412964722,
    14128.983605851688,
    6922.037069460559,
    10304.904051788348,
    5173.929445415735,
    3709.4811740062432,
    12552.68419863264,
    5806.847741854566,
    12405.367073509733,
    9563.81637260272,
    5952.683022688187,
    4821.51570294406,
    17257.539785073797,
    7076.868244008081,
    3020.42114327717,
    6406.43486506171,
    7619.97423134304,
    8973.363343824043,
    8456.301865430192,
    7348.5706519632295,
    6057.41507811255,
    6747.883159860408,
    5605.921958849889,
    5153.19429371538,
    3200.991970941235,
    7818.576949875467,
    18037.93430488753,
    8403.433523143663,
    3270.046995628494,
    5371.652209947781,
    5244.149953635919,
    6307.150761881581,
    10631.39030238799,
    13291.251312772962,
    6338.421613749037
  ],
  "Lasso": [
    10137.904848762184,
    9095.376114789113,
    4493.103863441065,
    14558.649801952575,
    2850.6197852377663,
    7919.965520481848,
    3963.1916571672773,
    6725.838846129744,
    5870.734906993195,
    6139.600165552663,
    7036.148716772295,
    21957.25942699612,
    12714.245801668476,
    5484.70437332381,
    8815.359489096969,
    5989.612645441237,
    8227.69994131555,
    2429.354843504465,
    3281.5320319081366,
    4012.719866621337,
    9489.965009089363,
    9455.889272503679,
    25663.991004601943,
    7539.103077223437,
    6497.8061315828745,
    12530.512570424304,
    8343.786604714776,
    5931.627415915995,
    5905.700838600875,
    7544.686720614474,
    6300.500387355845,
    13590.536103833489,
    5715.652146961169,
    13555.085714467335,
    13360.790825916934,
    6381.471852816954,
    4423.931882899924,
    21367.335155591107,
    4840.210404028963,
    10542.01961148029,
    7627.81541478727,
    17340.222014951167,
    9023.863810746276,
    8370.086197643868,
    5015.965181121537,
    19309.98748218111,
    6584.248195258588,
    5836.868905973533,
    8325.80811438631,
    12426.451671812167,
    5290.02602751302,
    7087.59691536599,
    5769.10577047593,
    5699.326952038697,
    32137.76112468391,
    10275.702715849104,
    5258.880604986727,
    8977.013330504466,
    3272.702491048716,
    2067.1409910761795,
    3621.8762892211234,
    3533.8057637306774,
    4957.144223823008,
    15885.323052568649,
    3227.524800968182,
    7339.450255051117,
    6078.010050522746,
    3370.33356915124,
    3744.521345377543,
    5244.215285414888,
    7214.1807874985025,
    7346.515893634532,
    11825.343167762938,
    5886.52469715185,
    3330.1515753193235,
    10822.952092432111,
    5055.710687357052,
    3919.7465499479586,
    9843.002569669428,
    3640.312665267804,
    14471.649183812013,
    4292.391906682868,
    6922.446291497501,
    6271.5590342976575,
    11652.762276586851,
    14272.118833150544,
    7205.426611475101,
    14128.821620067596,
    6921.963022235905,
    10304.953746583204,
    5173.731586957299,
    3709.5183873182687,
    12552.769152526114,
    5807.038559956105,
    12405.191854747492,
    9563.608567120573,
    5952.69794248652,
    4821.580003140123,
    17257.42563081062,
    7076.796454916206,
    3020.644138306685,
    6406.224469483219,
    7619.89997532063,
    8973.366489600077,
    8456.261593702591,
    7348.709144833279,
    6057.506723613413,
    6748.012074438237,
    5606.00268867259,
    5152.962218280722,
    3200.9259811073352,
    7818.627419719254,
    18037.365553218166,
    8403.50734008602,
    3270.0620399699765,
    5371.699544487041,
    5244.487285245479,
    6307.364440897458,
    10631.230866266815,
    13291.131570919504,
    6338.380930892818
  ]
}

In [ ]:
import numpy as np

for model, maes in results.items():
    mean_mae = np.mean(maes)
    print(f"{model}: mean MAE = {mean_mae:.2f}")

GBR: mean MAE = 8428.04
XGB: mean MAE = 8576.55
KNN: mean MAE = 8240.01
LGBM: mean MAE = 8482.64
Ridge: mean MAE = 8234.20
Lasso: mean MAE = 8234.18


: 